In [1]:
import os
import re
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from datetime import date

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from sklearn.metrics import mean_squared_error

In [4]:
from main_classes.DataRetriever import DataGetter, DataSearcher, load_npz
from main_classes.ResultDisplayer import (
    plot_metric_results, 
    plot_statistical_groups,
    plot_pred,
    plot_prediction,
    plot_mcdropout,
    plot_config_prediction
)
from utils.process_helpers import split_trials_from_scanpath
from main_classes.MetricsCalculator import calculate_multimatch, detect_sac_fix_from_scanpath, calculate_scanmatch

from utils.utils import merge_dicts

from utils.ScanMatchCalculator import ScanMatch_Struct, ScanMatch_FixationToSequence, ScanMatch 


In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
"""
posscan = "PosScan"
test_features_slopes = "test_features_slopes"
test_features_lin = "test_features_lin"
test_features_pl = "test_features_pl"
test_features_local_slopes = "test_features_local_slopes"
test_features_GL_slopes = "test_features_GL_slopes"
test_features_LC = "test_features_LC"
"""

'\nposscan = "PosScan"\ntest_features_slopes = "test_features_slopes"\ntest_features_lin = "test_features_lin"\ntest_features_pl = "test_features_pl"\ntest_features_local_slopes = "test_features_local_slopes"\ntest_features_GL_slopes = "test_features_GL_slopes"\ntest_features_LC = "test_features_LC"\n'

In [7]:
#model_name = "FovSOS-FS"
#model_name = "test_features_LC"
model_name = "FovSOS-FS_DirectPred"

RESULTS_FOLDER = f"../results/{model_name}/" #"../results/MCDropout/"
SAVE_FOLDER = f"../results_plots/{model_name}/"

dates_results = {
    "MCD": "28072021",
    "FovSOS-FS": "17112021",
    #"FovSOS-FS_DirectPred": "17112021"
    "FovSOS-FS_DirectPred": "25112021"
}

In [8]:
load_results = False
save_results = True
only_all_vs_all = True

if load_results:
    date_results = dates_results[model_name]
else:
    date_results = date.today().strftime("%d%m%Y")

In [9]:
date_results

'11072022'

In [10]:
#select_metrics = ["MSE", "peak_x", "peak_y", "dtw", "scanmatch", "rec", "multimatch"]
select_metrics = ["MSE", "peak_x", "peak_y", "scanmatch", "multimatch"]
#select_metrics = ["MSE", "peak_x", "peak_y", "scanmatch"]
#select_metrics = ["scanmatch"]
#select_metrics = ["MSE"]

In [11]:
if not os.path.exists(f'../results/assets/{model_name}/'):
    os.makedirs(f'../results/assets/{model_name}/')

In [12]:
#y_real, y_pred = load_npz(
#     os.path.join(RESULTS_FOLDER,"SUBJECT_s620-tIMG_natural-N_20-Nt_1-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150"), 
#     "white")

In [13]:
if not only_all_vs_all:
    # Load Results
    if load_results:
        with open(f'../results/assets/{model_name}/results_all_{date_results}.json') as data_file: 
            all_results = json.load(data_file)
    else:
        data_getter = DataGetter(RESULTS_FOLDER)
        all_results = data_getter.calculate_metrics_results(only_all_vs_all=False, select_metrics=select_metrics)

    # Save Results
    if save_results:
        with open(f'../results/assets/{model_name}/results_all_{date_results}.json', 'w') as f: #results_dict_test
            json.dump(all_results, f)

In [14]:
#all_results["natural"]["SUBJECT_s605-tIMG_natural-N_20-Nt_1-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150"]

In [15]:
if only_all_vs_all:
    # Load Results
    if load_results:
        with open(f'../results/assets/{model_name}/results_all_vs_all_{date_results}.json') as data_file:
            results_all_vs_all = json.load(data_file)
    else:    
        data_getter = DataGetter(RESULTS_FOLDER)
        results_all_vs_all = data_getter.calculate_metrics_results(only_all_vs_all=True, select_metrics=select_metrics)

    # Save Results
    if save_results:
        with open(f'../results/assets/{model_name}/results_all_vs_all_{date_results}.json', 'w') as f: 
            json.dump(results_all_vs_all, f)

SUBJECT_s620-tIMG_inverted-N_1-Nt_5-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s613-tIMG_pink_noise-N_1-Nt_20-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s617-tIMG_white-N_1-Nt_5-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s609-tIMG_black-N_1-Nt_11-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s611-tIMG_white_noise-N_1-Nt_11-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s611-tIMG_grey-N_1-Nt_11-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s622-tIMG_grey-N_1-Nt_11-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s605-tIMG_grey-N_1-Nt_20-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s609-tIMG_grey-N_1-Nt_5-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s616-tIMG_white_noise-N_1-Nt_5-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s620-tIMG_pink_noise-N_1-Nt_20-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s613-tIMG_grey-N_1-Nt_11-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s605-tIMG_invert

===========> Loading npz data from natural images


/home/camilojd/anaconda3/envs/scanpath-env/lib/python3.7/site-packages/elephant/conversion.py:1169: UserWarning: Binning discarded 1 last spike(s) of the input spiketrain
  "input spiketrain".format(n_discarded))


===========> Loading npz data from natural images


/home/camilojd/anaconda3/envs/scanpath-env/lib/python3.7/site-packages/elephant/conversion.py:1169: UserWarning: Binning discarded 1 last spike(s) of the input spiketrain
  "input spiketrain".format(n_discarded))


...

===========> Loading npz data from natural images


/home/camilojd/anaconda3/envs/scanpath-env/lib/python3.7/site-packages/elephant/conversion.py:1169: UserWarning: Binning discarded 1 last spike(s) of the input spiketrain
  "input spiketrain".format(n_discarded))


===========> Loading npz data from natural images
===========> Loading npz data from natural images


/home/camilojd/anaconda3/envs/scanpath-env/lib/python3.7/site-packages/elephant/conversion.py:1169: UserWarning: Binning discarded 1 last spike(s) of the input spiketrain
  "input spiketrain".format(n_discarded))


===========> Loading npz data from natural images
SUBJECT_s617-tIMG_white-N_1-Nt_11-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s619-tIMG_inverted-N_1-Nt_11-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s617-tIMG_black-N_1-Nt_5-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s613-tIMG_white-N_1-Nt_5-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s611-tIMG_black-N_1-Nt_1-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s620-tIMG_pink_noise-N_1-Nt_5-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s611-tIMG_grey-N_1-Nt_1-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s622-tIMG_white_noise-N_1-Nt_20-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s609-tIMG_white_noise-N_1-Nt_5-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s609-tIMG_pink_noise-N_1-Nt_20-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s613-tIMG_black-N_1-Nt_5-SEQ_10-InUts_30-LR_0.0001-EPOCHS_5000-PAT_150
SUBJECT_s605-tIMG_grey-N_1-Nt_5-SEQ_10-InUts_30-LR_0.

In [16]:
"""
from utils.utils import merge_dicts
all_results_new = all_results.copy()
with open(f'../results/assets/{model_name}/results_all_17112021.json') as data_file: 
    all_results_old = json.load(data_file)
a_r = merge_dicts(all_results_old, all_results_new)
with open(f'../results/assets/{model_name}/results_all_25112021.json', 'w') as f: #results_dict_test
    json.dump(a_r, f)
"""

"\nfrom utils.utils import merge_dicts\nall_results_new = all_results.copy()\nwith open(f'../results/assets/{model_name}/results_all_17112021.json') as data_file: \n    all_results_old = json.load(data_file)\na_r = merge_dicts(all_results_old, all_results_new)\nwith open(f'../results/assets/{model_name}/results_all_25112021.json', 'w') as f: #results_dict_test\n    json.dump(a_r, f)\n"